#### Import the relevant libraries

In [1]:
from PIL import Image
from PIL import ImageDraw
import numpy as np
import copy
import time
import math

#### Read and convert to greyscale

Open the image and convert it to greyscale since we are asked to convolve on greyscale images

In [2]:
def display_np(x):
    """
    Display a numpy array as an image
    """
    result = Image.fromarray(x.astype('uint8'))
    result.show()

In [3]:
def check_range(x): return np.min(x), np.max(x)

In [4]:
def read_image(x):
    x = Image.open('test-images/' + x).convert("L")
    x = np.array(x)
    return x

In [5]:
def read_list(imgs):
    return [read_image(im) for im in imgs]

In [6]:
name = 'custom_test.jpeg'
pixels = read_image(name)
pixels.shape

(1600, 1237)

#### Padding the image

In [7]:
a = np.ones((3,3))
a

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [8]:
def pad_image(x, kernel):
    """
    Create a new numpy array with all values 255 and then add the image in between
    i/p: image, kernel
    o/p: padded_kernelx
    """
    to_add = kernel.shape[0] - 1
    padded_image = np.full((x.shape[0] + to_add, x.shape[1] + to_add), 255)
    t = to_add // 2
    padded_image[t: padded_image.shape[0] - t, t: padded_image.shape[1] - t] = x
    return padded_image

In [9]:
p = pad_image(pixels, a)
p.shape

(1602, 1239)

In [10]:
assert p.shape == (pixels.shape[0] + a.shape[0] - 1, pixels.shape[1] + a.shape[0] - 1)

In [11]:
a = np.zeros((3,3))
a[1,1] = 1


d = np.array([[0.003,0.013,0.022,0.013,0.003],
             [0.013,0.059,0.097,0.059,0.013],
             [0.022,0.097,0.159,0.097,0.022],
             [0.013,0.059,0.097,0.059,0.013],
             [0.003,0.013,0.022,0.013,0.003]])

e = np.zeros((5,5))
e[1:4,1:4] = a
alpha = 0.9 + 1
e = e * alpha - d

In [12]:
# display_np(pixels)

#### convolution

In [13]:
def conv2d(image, kernel, div = 1, clip = True):
    """
    Applies 2d convolution on a 2d image. Also works with rectangular kernels
    """

    # number of rows and columns of the kernel
    r = kernel.shape[0]
    c = kernel.shape[1]

    # initialize a canvas for the output with 255s. We will fill values in this
    output = np.full(image.shape, 255)
    for i in range(image.shape[0] - r - 1):
        for j in range(image.shape[1] - c - 1):
            output[i][j] = np.sum(kernel * image[i:i+r, j:j+c]) / div
    if clip: np.clip(output, 0, 255, out = output)
    return output
    #display_np(output)

In [14]:
sobel = np.array([[-1, 0, 1],
                 [-2, 0, 2],
                 [-1, 0, 1]])

In [15]:
# conv2d(p, sobel)

In [16]:
p = conv2d(p, e)

#### Template matching

In [17]:
templates = read_list(['template1.png', 'template2.png', 'template3.png'])

In [18]:
templates[0].shape

(11, 17)

In [19]:
canvas = np.full(p.shape, 255)

In [20]:
def find_threshold(y, i): return 0.89 if i == 0 else 0.97

In [21]:
def get_indexes(x, vals):
    idxs1, idxs2 = np.array([]), np.array([])
    for v in vals:
        v_idxs = np.where(x == v)
        idxs1 = np.append(idxs1 ,v_idxs[0])
        idxs2 = np.append(idxs2 ,v_idxs[1])
    return idxs1, idxs2

In [22]:
def temp_match(templates, padded_im, name):
    """
    Takes a list of templates and an image and returns the matched template
    """
    pix = Image.fromarray(padded_im.astype('uint8')).convert("RGB")
    img = ImageDraw.Draw(pix)  
    i = 0
    temp_names = ['note', 'quarter_rest', 'eighth rest']
    txt_op = []
    excluded = ['music2.png', 'music3.png', 'music4.png', 'rach.png']
    for template in templates:
        # apply the hamming distance forumula
        op = conv2d(padded_im, template, 1, False) + conv2d(255 - padded_im, 255 - template, 1, False)

        # get the min and maximum value
        x,y = check_range(op)
        op = np.divide(op, np.max(op))

        # find threshold, eg if max val is 583 thresh = 500
        thresh = find_threshold(y, i)
        if temp_names[i] == 'quarter_rest' and name in excluded:
            i += 1
            continue
        # get all the values above thresh
        temp_op = op[op > thresh]
        conf = np.append(temp_op, temp_op)

        # get indexes of the values above thresh
        idxs1, idxs2 = get_indexes(op, temp_op)
        conf = conf[0:len(idxs1)]

        tr, tc = int(template.shape[0]), int(template.shape[1])
        cr, cc = int(canvas.shape[0]), int(canvas.shape[1])

        # if the template is not going out of the canvas, plot it
        for r,c in zip(idxs1, idxs2):
            r, c = int(r), int(c)
            if r+tr < cr and c+tc < cc:
                canvas[r:r+tr, c:c+tc] = padded_im[r:r+tr, c:c+tc]
                txt_op.append([r,c,tr,tc,temp_names[i], 'A', round(conf[i],3)])
                if i == 0:
                    img.rectangle(((c-1,r-1),(c+tc-1,r+tr-1)),fill=None,outline="red")
                elif i == 1:
                    img.rectangle(((c-1,r-1),(c+tc-1,r+tr-1)),fill=None,outline="green")
                elif i == 2:
                    img.rectangle(((c-1,r-1),(c+tc-1,r+tr-1)),fill=None,outline="blue")
        i+=1
                    
    pix.show()
    pix.save('detected_' + name)

In [23]:
temp_match(templates,p, name)

In [24]:
# display_np(canvas)

In [25]:
5 / 0

ZeroDivisionError: division by zero

#### Part 6

In [ ]:
#Using sobel kernels
sobel_kernel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])

#Convolving the image
conv_img = conv2d(p, sobel_kernel.T)

conv_img.shape

#display_np(conv_img)
# print(pixels.shape)

In [ ]:
edge_maps = copy.deepcopy(conv_img)
edge_maps[edge_maps != 0] = 1

e2 = copy.deepcopy(edge_maps)

In [ ]:
#Creating a function to calculate distance
def calc_dist(a, b, c, d):
    
    #Calculating distance
    dist = np.sqrt(np.abs(((a - c)**2) + ((b - d)**2)))
    return dist

#Retrieving the number of rows and columns in the image
row, col = conv_img.shape

#Creating an array to store distances
D = []

#Iterating through the image pixels
for p in range(0,row):
    for q in range(0,col):
        
        #Creating a temporary array
        temp = []
        
        #Iterating through the edge pixels
        for i in range(0,row):
            for j in range(i,col):
                if(conv_img[i][j] == 255):
                    
                    #Calculating distance and storing it in temp
                    dist = calc_dist(p, q, i, j)
                    temp.append(dist)
                    
        #Converting the temporary array to a numpy array
        temp = np.array(temp)
        
        #Finding the smallest distance and storing it in D
        min_dist = np.min(temp)
        D.append(min_dist)
        
        print(p,q)
        
#Reshaping D to that of the image
D = np.array(D).reshape(conv_img.shape)

#### Hough transform

In [ ]:
conv_img.shape

In [ ]:
#Referred https://www.science-emergence.com/Articles/Implementing-a-simple-python-code-to-detect-straight-lines-using-Hough-transform/
import matplotlib.pyplot as plt
import scipy.ndimage.filters as filters
import scipy.ndimage as ndimage

#Creating a function to perform Hough Transform on the image
def hough_transform(img, threshold):
    
    #Finding the shape of the image
    img_shape = img.shape

    #Finding the number of rows and columns in the image
    nrow = img_shape[0]
    ncol = img_shape[1]

    #Finding the maximum and minimum value of theta (ranges from -pi/2 to pi/2 so a total of pi radians)
    theta_max = 1.0 * math.pi 
    theta_min = 0.0

    #Finding the maximum and minimum distance from the origin (ranges from 0 to the diagonal of the image)
    rho_min = 0.0
    rho_max = math.hypot(nrow, ncol)

    #Setting the rho and theta dimensions in the hough space
    rho_dim = 200 
    theta_dim = 300

    #Creating a hough space filled with zeros
    hough_space = np.zeros((rho_dim, theta_dim))
    
    #Iterating through the image
    for x in range(nrow):
        for y in range(ncol):
            
            #Checking if the pixel is an edge
            if img[x,y] == 255: continue
            
            #Iterating through the theta dimension
            for itheta in range(theta_dim):
                
                #Calculating the value of theta as float
                theta = 1.0 * itheta * theta_max / theta_dim
                
                #Calculating the value of rho using theta
                rho = x * math.cos(theta) + y * math.sin(theta)
                
                #Finding the rho iterator as float
                irho = rho_dim * ( 1.0 * rho ) / rho_max
                
                hough_space[int(irho), int(itheta)] = hough_space[int(irho), int(itheta)] + 1

    #Plotting the Hough Space
    plt.imshow(hough_space, origin='lower')
    plt.xlim(0, theta_dim)
    plt.ylim(0, rho_dim)

    tick_locs = [i for i in range(0,theta_dim,40)]
    tick_lbls = [round( (1.0 * i * theta_max) / theta_dim,1) for i in range(0, theta_dim, 40)]
    plt.xticks(tick_locs, tick_lbls)

    tick_locs = [i for i in range(0,rho_dim,20)]
    tick_lbls = [round( (1.0 * i * rho_max ) / rho_dim, 1) for i in range(0, rho_dim, 20)]
    plt.yticks(tick_locs, tick_lbls)

    plt.xlabel(r'Theta')
    plt.ylabel(r'Rho')
    plt.title('Hough Space')

    plt.savefig("test-images/hough_image.png", bbox_inches='tight')

    plt.close()
    
    #Setting the size of the neighborhood
    neighborhood_size = 30
    
    #Finding the multi-dimensionsal maximum filter of the hough space
    data_max = filters.maximum_filter(hough_space, neighborhood_size)

    #Finding the local maxima in the hough space
    maxima = (hough_space == data_max)

    #Finding the multi-dimensionsal minimum filter of the hough space
    data_min = filters.minimum_filter(hough_space, neighborhood_size)

    #
    diff = ((data_max - data_min) > threshold)
    maxima[diff == 0] = 0

    #Finding the number of features in the local maxima
    labeled, num_objects = ndimage.label(maxima)

    #Finding the features in the labeled array
    slices = ndimage.find_objects(labeled)

    #Creating temporary arrays to store the cooridnates of the feature centers in the hough space
    x, y = [], []

    #Iterating through the features
    for dy,dx in slices:

        #Finding the x coordinate of the center index of the feature and appending to the array
        x_center = (dx.start + dx.stop - 1)/2
        x.append(x_center)

        #Finding the y coordinate of the center index of the feature and appending to the array
        y_center = (dy.start + dy.stop - 1)/2    
        y.append(y_center)

    #Printing the centers
    print(x)
    print(y)

    #Plotting the Hough Space
    plt.imshow(hough_space, origin = 'lower')
    plt.savefig('test-images/hough_space_i_j.png', bbox_inches = 'tight')

    plt.autoscale(False)
    plt.plot(x,y, 'ro')
    plt.savefig('test-images/hough_space_maximas.png', bbox_inches = 'tight')

    plt.close()
    
    #Initializing the line index
    line_index = 1

    #Iterating through the feature centers
    for i,j in zip(y, x):

        #Finding the values of rho and theta
        rho = round( (1.0 * i * rho_max ) / rho_dim, 1)
        theta = round( (1.0 * j * theta_max) / theta_dim, 1)

        fig, ax = plt.subplots()

        #Creating a copy of the original image
        copy_img = np.copy(img)

        #Plotting the copied image
        ax.imshow(copy_img)
        ax.autoscale(False)

        #Creating temporary arrays to store points
        px = []
        py = []

        #Converting points from polar coordinates to cartesian coordinates
        for i in range(-ncol - 40, ncol + 40, 1):
            px.append( math.cos(-theta) * i - math.sin(-theta) * rho ) 
            py.append( math.sin(-theta) * i + math.cos(-theta) * rho )

        #PLotting the cartesian points
        ax.plot(px, py, linewidth = 10)

        plt.savefig("test-images/image_line_"+ "%02d" % line_index +".png", bbox_inches='tight')

        plt.close()

        #Incrementing the line index
        line_index = line_index + 1

In [ ]:
hough_transform(conv_img, 100)

#### 1d convolution

In [ ]:
def conv1d(image, kernel, div = 1):
    """
    Performs 1d convolution on an image
    """
    
    im = copy.deepcopy(image.flatten())
    k = copy.deepcopy(kernel.flatten())

    size = len(k)
    to_remove = len(k) - 1
    
    output = np.full((im.shape), 255)
    for i in range(len(im) - to_remove):
        output[i] = np.sum(k * im[i:i+size]) / div
    return output

In [ ]:
h1 = np.array([1,2,1])
h2 = np.array([-1,0,1])

In [ ]:
tmp = conv1d(p, h1)

In [ ]:
result = conv1d(tmp, h2)

In [ ]:
check_range(result)

In [ ]:
np.clip(result, 0, 255, out = result)
check_range(result)

In [ ]:
result = result.reshape(p.shape)
result.shape

# Fin